<a href="https://colab.research.google.com/github/Vidhu-Thalla/new-repo/blob/main/1_TM_real_time_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1:  Initializations:**

- From Teachable Machine (TM), download your model:   
  - Load your model into TM, rerun the Training step
  - From Preview:  Export Model - TensorFlow - OpenCVKeras (will generate keras_model.h5 and labels.txt)
  - Copy files to your Google drive
- Mount Google Drive
- cd to the folder your model was uploaded to
- Import Packages



In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Curriculum/MachineLearning/COURSE MATERIAL/GOOGLE CLASSROOM/Machine Learning Foundations (MLF)/MLF_Unit-100 - Intro to Teachable Machine and Google Colab'
!pwd

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/Curriculum/MachineLearning/COURSE MATERIAL/GOOGLE CLASSROOM/Machine Learning Foundations (MLF)/MLF_Unit-100 - Intro to Teachable Machine and Google Colab'
/content
/content


In [2]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

**Step 2:  "Predict" function:**   

Input numpy array containing an image and outputs prediction using your TM model.

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

class CustomDepthwiseConv2D(layers.DepthwiseConv2D):
    def __init__(self, *args, groups=None, **kwargs):
        # Remove the groups argument if present
        super().__init__(*args, **kwargs)

# Register the custom layer
custom_objects = {
    'DepthwiseConv2D': CustomDepthwiseConv2D
}

# Now load the model with the custom objects
model = tf.keras.models.load_model("keras_model.h5",
                                 custom_objects=custom_objects,
                                 compile=False)
# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)




def predict(image):

  # resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

  # Normalize the image
  normalized_image_array = (image.astype(np.float32) / 127.5) - 1

  # Load the image into the array
  data[0] = normalized_image_array


  # Predicts the model
  prediction = model.predict(data, verbose = 0)
  index = np.argmax(prediction)
  class_name = class_names[index]
  confidence_score = prediction[0][index]

  # Print prediction and confidence score
  return class_name[2:], confidence_score




FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'keras_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

**Step 3:  Convert from Javascript to OpenCV:**  

Functions to take the data from the javascript frame and convert to OpenCV image (**js_to_image**) and the other way around as well (**bbox_to_bytes**)

In [ ]:
# Convert the JavaScript object into an OpenCV image

def js_to_image(js_reply):
 # Input:   js_reply: JavaScript object containing image from webcam
 # Returns: img: OpenCV BGR image

  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def bbox_to_bytes(bbox_array):
# Input:   bbox_array: Numpy array (pixels)
#          containing rectangle to overlay on video stream.
# Returns: bytes: Base64 image byte string

  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()),
                                        'utf-8')))

  return bbox_bytes



**Step 4:  Stream live video**

From webcam w/ javascript

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {
              'img': result};
    }
    ''')

  display(js)

first = 0
def video_frame(label, bbox):
  global first
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  first = first+1
  return data

**Step 5:  Run predictions and write classification label on screen**

In [ ]:
# start streaming video from webcam'
import time
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # call our darknet helper on video frame

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    label1, score1 = predict(frame)
    label1 = label1[:-1]
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    org2 = (50, 100)
    fontScale = 1
    color = (255, 0, 0)
    color2 = (0, 0, 255)
    thickness = 2

    bbox_array = cv2.putText(bbox_array, label1, org, font,
                   1.5, color, thickness, cv2.LINE_AA)
    bbox_array = cv2.putText(bbox_array, str(score1), org2, font,
                   1, color2, thickness, cv2.LINE_AA)


    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes
